# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [2]:
# Set the file path to import the WeatherPy_database.csv file
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,80.76,74,75,12.66,scattered clouds
1,1,Dikson,RU,73.5069,80.5464,31.82,99,100,5.12,scattered clouds
2,2,Hobart,AU,-42.8794,147.3294,55.26,59,40,13.80,scattered clouds
3,3,Albany,US,42.6001,-73.9662,48.45,79,100,4.14,scattered clouds
4,4,Mataura,NZ,-46.1927,168.8643,41.79,99,99,6.13,scattered clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [3]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [4]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp) ]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,80.76,74,75,12.66,scattered clouds
6,6,Veraval,IN,20.9000,70.3667,81.27,74,94,6.87,scattered clouds
16,16,Moussoro,TD,13.6408,16.4901,83.25,52,56,8.61,scattered clouds
17,17,San Policarpo,PH,12.1791,125.5072,85.24,69,74,7.76,scattered clouds
18,18,Georgetown,MY,5.4112,100.3354,88.90,72,20,3.44,scattered clouds


In [5]:
len(preferred_cities_df)

179

## No empty rows.

In [6]:
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
preferred_cities_df.notnull().sum()

City_ID                179
City                   179
Country                179
Lat                    179
Lng                    179
Max Temp               179
Humidity               179
Cloudiness             179
Wind Speed             179
Current Description    179
dtype: int64

In [8]:
preferred_cities_df.count()

City_ID                179
City                   179
Country                179
Lat                    179
Lng                    179
Max Temp               179
Humidity               179
Cloudiness             179
Wind Speed             179
Current Description    179
dtype: int64

## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [9]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description","Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,80.76,scattered clouds,-21.2000,-175.2000,
6,Veraval,IN,81.27,scattered clouds,20.9000,70.3667,
16,Moussoro,TD,83.25,scattered clouds,13.6408,16.4901,
17,San Policarpo,PH,85.24,scattered clouds,12.1791,125.5072,
18,Georgetown,MY,88.90,scattered clouds,5.4112,100.3354,
21,Merauke,ID,87.24,scattered clouds,-8.4667,140.3333,
25,Kahului,US,80.62,scattered clouds,20.8947,-156.4700,
29,Avarua,CK,77.05,scattered clouds,-21.2078,-159.7750,
32,Butaritari,KI,82.18,scattered clouds,3.0707,172.7902,
35,Fuling,CN,78.94,scattered clouds,29.7022,107.3919,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [10]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [11]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found... skipping.")
        pass

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [13]:
hotel_df.count()

City                   179
Country                179
Max Temp               179
Current Description    179
Lat                    179
Lng                    179
Hotel Name             179
dtype: int64

In [14]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,80.76,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
6,Veraval,IN,81.27,scattered clouds,20.9000,70.3667,Lords Inn Somnath
16,Moussoro,TD,83.25,scattered clouds,13.6408,16.4901,Pharmacie alnadja Moussoro
17,San Policarpo,PH,85.24,scattered clouds,12.1791,125.5072,Agapita M. Nicart
18,Georgetown,MY,88.90,scattered clouds,5.4112,100.3354,Cititel Penang
21,Merauke,ID,87.24,scattered clouds,-8.4667,140.3333,
25,Kahului,US,80.62,scattered clouds,20.8947,-156.4700,Maui Seaside Hotel
29,Avarua,CK,77.05,scattered clouds,-21.2078,-159.7750,Paradise Inn
32,Butaritari,KI,82.18,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
35,Fuling,CN,78.94,scattered clouds,29.7022,107.3919,Yingbin Hostel


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [15]:
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,80.76,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
6,Veraval,IN,81.27,scattered clouds,20.9000,70.3667,Lords Inn Somnath
16,Moussoro,TD,83.25,scattered clouds,13.6408,16.4901,Pharmacie alnadja Moussoro
17,San Policarpo,PH,85.24,scattered clouds,12.1791,125.5072,Agapita M. Nicart
18,Georgetown,MY,88.90,scattered clouds,5.4112,100.3354,Cititel Penang
25,Kahului,US,80.62,scattered clouds,20.8947,-156.4700,Maui Seaside Hotel
29,Avarua,CK,77.05,scattered clouds,-21.2078,-159.7750,Paradise Inn
32,Butaritari,KI,82.18,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
35,Fuling,CN,78.94,scattered clouds,29.7022,107.3919,Yingbin Hostel
41,Ko Samui,TH,82.44,scattered clouds,9.5357,99.9357,The Passage


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [16]:
# Set the file name.
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [17]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [18]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>Keleti Beach Resort</dd>\n<dt>City</dt><dd>Vaini</dd>\n<dt>Country</dt><dd>TO</dd>\n<dt>Current Weather</dt><dd>scattered clouds and 80.76 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Lords Inn Somnath</dd>\n<dt>City</dt><dd>Veraval</dd>\n<dt>Country</dt><dd>IN</dd>\n<dt>Current Weather</dt><dd>scattered clouds and 81.27 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Pharmacie alnadja Moussoro</dd>\n<dt>City</dt><dd>Moussoro</dd>\n<dt>Country</dt><dd>TD</dd>\n<dt>Current Weather</dt><dd>scattered clouds and 83.25 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Agapita M. Nicart</dd>\n<dt>City</dt><dd>San Policarpo</dd>\n<dt>Country</dt><dd>PH</dd>\n<dt>Current Weather</dt><dd>scattered clouds and 85.24 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Cititel Penang</dd>\n<dt>City</dt><dd>Georgetown</dd>\n<dt>Country</dt><dd>MY</dd>\n<dt>Current Weather</dt><dd>scattered clouds and 88.9 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [19]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
0,-21.2000,-175.2000
6,20.9000,70.3667
16,13.6408,16.4901
17,12.1791,125.5072
18,5.4112,100.3354
25,20.8947,-156.4700
29,-21.2078,-159.7750
32,3.0707,172.7902
35,29.7022,107.3919
41,9.5357,99.9357


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [20]:
#Reusing code 
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)

#Adding markers for each city on top of the heatmap
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))